In [313]:
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from itertools import combinations

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer 
from string import punctuation
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer 
from string import punctuation

from pymystem3 import Mystem

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import random 

import tensorflow as tf
from tensorflow.python import keras

from tensorflow.python.keras.layers import Input, Dense, Dropout, LSTM, Flatten, RepeatVector, Conv1D, MaxPool1D, MaxPooling1D
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.optimizer_v2.adam import Adam


from tensorflow.keras.layers.experimental import preprocessing

tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


Суть нижепроисходящего.

Мы берём размеченные 14 предложений и вбрасываем их в поток новостей за последние несколько часов.

Смотрим как хорошо пары смогут найти друг друга. 

Используются две модели трансформеров и простой бег-оф-вордс в качестве референса. 

Для работы ноутбука требуется compiled.csv который лежит здесь же. 

In [314]:
def ru_clear(args, is_stopword = True, is_stemm = False, is_lemma = False):
    stemm = SnowballStemmer("russian") 
    mystem = Mystem()
    
    symbols = "~!@#$?><%^&*)(_+=-*|/"
    punct = ",.'`;:"
    stoplist = ['\xa0130', '\xa065', '\xa0', '\n', '"', '«', '»'  ]
    new_list = []
    temp = str()
    
    russian_stopwords = stopwords.words("russian")
    for arg in args:
        for symbol in symbols:
            temp = arg[0].replace(symbol, "")
        for pun in punct:
            temp = arg[0].replace(pun, "")   
        for stop in stoplist:
             temp = arg[0].replace(stop, " ")         
        if is_lemma == True:
            tokens = mystem.lemmatize(temp.lower())
        else:
            tokens = nltk.word_tokenize(temp.lower())
        
        if is_stopword:
            tokens = [token.strip() for token in tokens if token not in russian_stopwords and token !=' ' and token not in punctuation]
        else: 
            tokens = [token.strip() for token in tokens if token !=' ' and token not in punctuation]
        
        line = str()
        for token in tokens:
            if is_stemm == True:
                token = stemmer.stem(token)
            line += token.strip() + " "
        new_list.append((line[:-1], arg[1]))

    return new_list 

In [315]:
def evaluator(text): 
    scors = []
    combos = combinations(list(range(len(text))), 2)
    unsorted_dict = {}
    bingo = 0
    count_vectorizer = CountVectorizer(stop_words='english', analyzer="word", ngram_range=(1,2), max_df=2) 
    
    for combo in combos:
        _input = [text[combo[0]][0], text[combo[1]][0]]
#         if tags[arg[0]] == tags[arg[1]]:
#             match = 1
#         else:
#             match = 0

        some_matrix = count_vectorizer.fit_transform(_input).todense()
        product = cosine_similarity(some_matrix)
        scors.append(
            ((combo[0], combo[1]), [np.sum(product)], text[combo[0]][0], text[combo[0]][1], 
                       "      MIDDLE    ", text[combo[1]][0], text[combo[1]][1], "     END   ")
                     )
    return scors


In [316]:
def result(text):
    scores_mirrored = []
    scores_mirrored = text
    temp = []
    for i in scores_mirrored:
        couple = list(i[0])
        couple[1], couple[0] = couple[0], couple[1]
        temp.append(((couple[0], couple[1]), i[1], i[5], i[6], i[4], i[2], i[3], i[7]))
 
    scores_mirrored = temp.copy()
    text =  scores_mirrored + text.copy()
    
    top_list = []
    scored_list = sorted(text, key=lambda x: (x[0][0], -x[1][0]))
    top_scored = scored_list[::len(data_set)-1]
    count = 0
    for top in top_scored:
        if top[6] != 0 and top[3] != 0:           
#            print(top)
            count += 1
            print(top)
#         count += 1
    print(" ")
    print(f'Точность - {count/ len(txt) * 100 } %')
    print(f'Обнаружено {count} пар из {len(txt)}')
    return scored_list




In [317]:
def hugging_embeddings_and_score(text):
    scores = []
    combos = combinations(list(range(len(text))), 2)
    
    for index, combo in enumerate(combos):
        left_score = model_st.encode(text[combo[0]][0])
        right_score = model_st.encode(text[combo[1]][0]) 
        score = cosine_similarity(left_score.reshape(1, -1), right_score.reshape(1, -1))                                       
        scores.append(((combo[0], combo[1]), score, text[combo[0]][0], text[combo[0]][1], "      MIDDLE    ", 
                      text[combo[1]][0], text[combo[1]][1], "     END   "))

    return scores
    

In [318]:
def compare_texts(text1, text2, combo):
#    text1= 
    batch = tokenizer(text1[0], text2[0], return_tensors='pt',  truncation=True, max_length=512).to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).cpu().numpy()
        
    return ((combo[0], combo[1]), [proba[0][1]], text1[0], text1[1], "      MIDDLE    ", text2[0], text2[1], "     END   ") # p(non-paraphrase), p(paraphrase)


In [319]:

def combinating_and_scoring(text): 
    scores = []
    combos = combinations(list(range(len(text))), 2)
    titles = []
    
    for index, combo in enumerate(combos):
        score = compare_texts(text[combo[0]], text[combo[1]], combo)
        
        scores.append(score)
    return scores

In [320]:
# Установка модели определения парафраза 
import torch
from transformers import AutoModelForSequenceClassification, BertTokenizer
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)


In [321]:
# Установка модели 

#!pip install sentence-transformers 
#!pip3 install pytorch-transformers

from sentence_transformers import SentenceTransformer
model_st = SentenceTransformer('distiluse-base-multilingual-cased')

In [322]:
txt='''Патрушев пригрозил Литве жестким ответом на запрет транзита
Россия, безусловно, ответит на действия Литвы и ЕС по ограничению транзита в Калининградскую область — Патрушев
Патрушев предупредил Литву об ответных мерах за блокаду транзита в Калининград
В Минобороны заявили о безумной попытке Украины захватить Змеиный
Минобороны сообщило о провале новой попытки захвата острова Змеиный Украиной
Средства ПВО России отбили атаку ВСУ на остров Змеиный
Первое иностранное судно «Азов Конкорд» вышло из порта Мариуполя
Первое иностранное судно покинуло порт Мариуполя
Девять резервистов ВСУ сдались в плен под Северодонецком
Девять украинских резервистов сдались в плен под Сиротино в пригороде Северодонецка
Девять украинских резервистов сдались в плен в пригороде Северодонецка
Республиканцы в Техасе хотят вывести штат из состава США
Развал США начинается. Техас готовится к независимости
Техас может выйти из состава США'''
txt = txt.split('\n')
ru_label = [1]*3 + [2]*3 + [3] + [3] + [4]*3 + [5]*3
ru_dict = {1: [0, 1, 2], 2: [3, 4, 5], 3: [6, 7], 4: [8, 9, 10], 5: [11, 12, 13]}

In [323]:
# Делаем привязку заголовков к меткам их групп
marked_lines = [(title, ru_label[value]) for value, title in enumerate(txt)]


In [324]:
compiled_df = pd.read_csv('compiled.csv', index_col=[0])
compiled_df.sort_values(by="time", inplace=True)
compiled_df.tail()

,article,name,picture,tag,time,title,url
101,Актер Майкл Джей Фокс получит почетный «Оскар»...,lenta-ru,https://icdn.lenta.ru/images/2022/06/23/14/202...,Культура,2022-06-23 16:23:00,Майкл Джей Фокс получит почетный «Оскар»,https://lenta.ru/news/2022/06/23/jfox/>
264,Великобританию ждет ухудшение продовольственн...,ria-ru,https://cdnn21.img.ria.ru/images/155568/98/155...,"Экономика,Россия,Великобритания,Украина",2022-06-23 16:23:00,В Британии готовятся к худшему после разрыва с...,https://ria.ru/20220623/britaniya-1797616138.html
102,"Путин заявил, что лидерство стран БРИКС востре...",lenta-ru,https://icdn.lenta.ru/images/2022/06/23/16/202...,Мир,2022-06-23 16:24:00,Путин указал на востребованность лидерства стр...,https://lenta.ru/news/2022/06/23/briks_lidersh...
103,Каспийский трубопроводный консорциум выплатит ...,lenta-ru,https://icdn.lenta.ru/images/2022/06/23/16/202...,Среда обитания,2022-06-23 16:24:00,За разлив нефти в Черном море выплатят миллиар...,https://lenta.ru/news/2022/06/23/ktk/>
104,Президент Украины Зеленский заявил об усталост...,lenta-ru,https://icdn.lenta.ru/images/2022/06/23/16/202...,Бывший СССР,2022-06-23 16:24:00,Зеленский заявил об усталости от темы войны на...,https://lenta.ru/news/2022/06/23/tired/>


In [325]:
# Выберите количество элементов из датасета.

# Выбрать по количеству последних записей в таблице.

example = compiled_df.tail(10)

# Выбрать интервал в часах от времени публикации самой свежей новости в сете.

interval_time = datetime.strptime(compiled_df['time'].max(), '%Y-%m-%d %H:%M:%S') - timedelta(hours = 4)

# example = compiled_df.loc[compiled_df["time"] >= interval_time.strftime('%Y-%m-%d %H:%M:%S')]['time']

In [326]:
# Определяем выборку из данных и сливаем её с размеченным текстом. 
ex = list(example.title)
ex = [(title, 0) for value, title in enumerate(ex) if len(title) > 1]
data_set = marked_lines.copy()
data_set.extend(ex)

In [327]:
# Метод очистки текста. По умолчанию убираются только стоп слова. Для стеммизации подайте параметр is_stemm = True .
# Для лемматизации подайте параметр is_lemma = True . 

ru_text= ru_clear(data_set) 


In [328]:
# Данные получены при значениях compiled_df.tail(50) 

Оцениваем модель трансформер 'cointegrated/rubert-base-cased-dp-paraphrase-detection'

In [329]:
paraphrase_model = combinating_and_scoring(ru_text) 

In [330]:
result(paraphrase_model)

((0, 2), [0.9787837], 'патрушев пригрозил литве жестким ответом запрет транзита', 1, '      MIDDLE    ', 'патрушев предупредил литву ответных мерах блокаду транзита калининград', 1, '     END   ')
((1, 2), [0.9742305], 'россия безусловно ответит действия литвы ес ограничению транзита калининградскую область — патрушев', 1, '      MIDDLE    ', 'патрушев предупредил литву ответных мерах блокаду транзита калининград', 1, '     END   ')
((2, 0), [0.9787837], 'патрушев предупредил литву ответных мерах блокаду транзита калининград', 1, '      MIDDLE    ', 'патрушев пригрозил литве жестким ответом запрет транзита', 1, '     END   ')
((3, 4), [0.8704645], 'минобороны заявили безумной попытке украины захватить змеиный', 2, '      MIDDLE    ', 'минобороны сообщило провале новой попытки захвата острова змеиный украиной', 2, '     END   ')
((4, 3), [0.8704645], 'минобороны сообщило провале новой попытки захвата острова змеиный украиной', 2, '      MIDDLE    ', 'минобороны заявили безумной попытке 

[((0, 2),
  [0.9787837],
  'патрушев пригрозил литве жестким ответом запрет транзита',
  1,
  '      MIDDLE    ',
  'патрушев предупредил литву ответных мерах блокаду транзита калининград',
  1,
  '     END   '),
 ((0, 1),
  [0.9590774],
  'патрушев пригрозил литве жестким ответом запрет транзита',
  1,
  '      MIDDLE    ',
  'россия безусловно ответит действия литвы ес ограничению транзита калининградскую область — патрушев',
  1,
  '     END   '),
 ((0, 19),
  [0.054317575],
  'патрушев пригрозил литве жестким ответом запрет транзита',
  1,
  '      MIDDLE    ',
  'майкл джей фокс получит почетный « оскар',
  0,
  '     END   '),
 ((0, 14),
  [0.053578973],
  'патрушев пригрозил литве жестким ответом запрет транзита',
  1,
  '      MIDDLE    ',
  'сайт спч атаковали хакеры',
  0,
  '     END   '),
 ((0, 16),
  [0.053536467],
  'патрушев пригрозил литве жестким ответом запрет транзита',
  1,
  '      MIDDLE    ',
  'международные резервы россии неделю сократились 2,1',
  0,
  '     E

In [331]:
# Точность - 100.0 %
# Обнаружено 14 пар из 14

# Примечание:
# Пара (5, 3) имеет скор 0,23.
# Но это наибольший скор 5го элемента среди других его пар. 

In [332]:
#    Напоминание в середине ноутбука 
#    Индексы:
#    0, 1, 2    - кластер 1
#    3, 4, 5    - кластер 2
#    6, 7       - кластер 3
#    8, 9, 10   - кластер 4 
#    11, 12, 13 - кластер 5 
           


Оцениваем модель трансформер 'distiluse-base-multilingual-cased' 

In [333]:
embedding_transformer = hugging_embeddings_and_score(data_set)

In [334]:
result(embedding_transformer )

((0, 2), array([[0.7960892]], dtype=float32), 'Патрушев пригрозил Литве жестким ответом на запрет транзита', 1, '      MIDDLE    ', 'Патрушев предупредил Литву об ответных мерах за блокаду транзита в Калининград', 1, '     END   ')
((1, 2), array([[0.6816091]], dtype=float32), 'Россия, безусловно, ответит на действия Литвы и ЕС по ограничению транзита в Калининградскую область — Патрушев', 1, '      MIDDLE    ', 'Патрушев предупредил Литву об ответных мерах за блокаду транзита в Калининград', 1, '     END   ')
((2, 0), array([[0.7960892]], dtype=float32), 'Патрушев предупредил Литву об ответных мерах за блокаду транзита в Калининград', 1, '      MIDDLE    ', 'Патрушев пригрозил Литве жестким ответом на запрет транзита', 1, '     END   ')
((3, 4), array([[0.8444619]], dtype=float32), 'В Минобороны заявили о безумной попытке Украины захватить Змеиный', 2, '      MIDDLE    ', 'Минобороны сообщило о провале новой попытки захвата острова Змеиный Украиной', 2, '     END   ')
((4, 3), array([

[((0, 2),
  array([[0.7960892]], dtype=float32),
  'Патрушев пригрозил Литве жестким ответом на запрет транзита',
  1,
  '      MIDDLE    ',
  'Патрушев предупредил Литву об ответных мерах за блокаду транзита в Калининград',
  1,
  '     END   '),
 ((0, 1),
  array([[0.6029059]], dtype=float32),
  'Патрушев пригрозил Литве жестким ответом на запрет транзита',
  1,
  '      MIDDLE    ',
  'Россия, безусловно, ответит на действия Литвы и ЕС по ограничению транзита в Калининградскую область — Патрушев',
  1,
  '     END   '),
 ((0, 5),
  array([[0.43261817]], dtype=float32),
  'Патрушев пригрозил Литве жестким ответом на запрет транзита',
  1,
  '      MIDDLE    ',
  'Средства ПВО России отбили атаку ВСУ на остров Змеиный',
  2,
  '     END   '),
 ((0, 3),
  array([[0.41532853]], dtype=float32),
  'Патрушев пригрозил Литве жестким ответом на запрет транзита',
  1,
  '      MIDDLE    ',
  'В Минобороны заявили о безумной попытке Украины захватить Змеиный',
  2,
  '     END   '),
 ((0, 23),

In [335]:
# Точность - 92.85 %
# Обнаружено 13 пар из 14

Оцениваем модель Bag of words при помощи библиотек sklearn: CountVectorizer и cosine_similarity

In [336]:
simple_bag_of_words = evaluator(ru_text)

In [337]:
result(simple_bag_of_words)

((0, 2), [2.286445949615773], 'патрушев пригрозил литве жестким ответом запрет транзита', 1, '      MIDDLE    ', 'патрушев предупредил литву ответных мерах блокаду транзита калининград', 1, '     END   ')
((1, 0), [2.242091013067522], 'россия безусловно ответит действия литвы ес ограничению транзита калининградскую область — патрушев', 1, '      MIDDLE    ', 'патрушев пригрозил литве жестким ответом запрет транзита', 1, '     END   ')
((2, 0), [2.286445949615773], 'патрушев предупредил литву ответных мерах блокаду транзита калининград', 1, '      MIDDLE    ', 'патрушев пригрозил литве жестким ответом запрет транзита', 1, '     END   ')
((3, 4), [2.2690691175985256], 'минобороны заявили безумной попытке украины захватить змеиный', 2, '      MIDDLE    ', 'минобороны сообщило провале новой попытки захвата острова змеиный украиной', 2, '     END   ')
((4, 3), [2.2690691175985256], 'минобороны сообщило провале новой попытки захвата острова змеиный украиной', 2, '      MIDDLE    ', 'миноборо

[((0, 2),
  [2.286445949615773],
  'патрушев пригрозил литве жестким ответом запрет транзита',
  1,
  '      MIDDLE    ',
  'патрушев предупредил литву ответных мерах блокаду транзита калининград',
  1,
  '     END   '),
 ((0, 1),
  [2.242091013067522],
  'патрушев пригрозил литве жестким ответом запрет транзита',
  1,
  '      MIDDLE    ',
  'россия безусловно ответит действия литвы ес ограничению транзита калининградскую область — патрушев',
  1,
  '     END   '),
 ((0, 3),
  [2.000000000000001],
  'патрушев пригрозил литве жестким ответом запрет транзита',
  1,
  '      MIDDLE    ',
  'минобороны заявили безумной попытке украины захватить змеиный',
  2,
  '     END   '),
 ((0, 7),
  [2.000000000000001],
  'патрушев пригрозил литве жестким ответом запрет транзита',
  1,
  '      MIDDLE    ',
  'первое иностранное судно покинуло порт мариуполя',
  3,
  '     END   '),
 ((0, 8),
  [2.000000000000001],
  'патрушев пригрозил литве жестким ответом запрет транзита',
  1,
  '      MIDDLE   

In [338]:
# Точность - 92.85 %
# Обнаружено 13 пар из 14